In [1]:
import os
import collections
import requests
# from dotenv import load_dotenv

# load_dotenv()
API_KEY = 'REDACTED'

#I was wholly unsucessfully in getting python-dotenv to work here. I gave up after spending admittedly 
#more time on it than I really could afford to. So for now the API_KEY is just going to remain defined in
#source.

print(API_KEY)

REDACTED


In [2]:
def requester(
    address = 'https://data.nasdaq.com/api/v3/datasets/',
    database_code = 'FSE',
    dataset_code= 'AFX_X',
    return_format = 'json',
    call_type = 'data',
    api_key = API_KEY,
    **optional_params):
   
    optionals = ''
    if optional_params:
        print(optional_params)
        for param in optional_params.keys():
            optionals = f'?{param}={optional_params[param]}' + optionals
   
    query = f'{optionals}{call_type}.{return_format}?api_key={api_key}'
    url = os.path.join(address, database_code, dataset_code, query)

    print(url)
    with requests.get(url) as response:
        return response

In [3]:
#NEED TO PULL DATA FOR PRECISELY THE YEAR 2017
response = requester(
    call_type = 'data',
    start_date='2017-01-02',
    end_date='2017-12-31')
    
json_data = response.json()

{'start_date': '2017-01-02', 'end_date': '2017-12-31'}
https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/?end_date=2017-12-31?start_date=2017-01-02data.json?api_key=REDACTED


In [6]:
#I'm being instructed not to use pandas dataframes here. I really don't get why. I am more than familiar 
#with dictionaries as a data structure, so I will code with them at the end
# of this document - but for now I'm going to use dataframes. Pandas seems to be the industry standard.

import pandas as pd, numpy as np

df = pd.DataFrame(
    json_data['dataset']['data'],
    columns = json_data['dataset']['column_names'])

In [7]:
final_idx = (df.loc[df.Date=='2016-12-30']).index[0]

df = df[:final_idx]

df

,Date,Open,High,Low,Close,Change,Traded Volume,Turnover,Last Price of the Day,Daily Traded Units,Daily Turnover
0,2017-12-29,51.76,51.94,51.45,51.76,NaN,34640.0,1792304.0,None,None,NaN
1,2017-12-28,51.65,51.82,51.43,51.60,NaN,40660.0,2099024.0,None,None,NaN
2,2017-12-27,51.45,51.89,50.76,51.82,NaN,57452.0,2957018.0,None,None,NaN
3,2017-12-22,51.05,51.50,50.92,51.32,NaN,71165.0,3641949.0,None,None,NaN
4,2017-12-21,51.16,51.52,50.90,51.40,NaN,120649.0,6179433.0,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...
250,2017-01-06,34.91,35.21,34.91,35.04,NaN,27507.0,964046.0,None,None,NaN
251,2017-01-05,35.02,35.20,34.73,35.06,NaN,48412.0,1692326.0,None,None,NaN
252,2017-01-04,35.48,35.51,34.75,35.19,NaN,54408.0,1906810.0,None,None,NaN
253,2017-01-03,35.90,35.93,35.34,35.48,NaN,70618.0,2515473.0,None,None,NaN


In [12]:
highest = df['High'].max()

print('highest:', highest) #Highest in 2017

lowest = df['Low'].min()

print('lowest:', lowest) #Lowest in 2017

deltas = df['High'] - df['Low']

max_change = deltas.max()

print('Max Change:', max_change)

next_close = df['Close'][1:].reset_index().drop(columns='index')['Close']

single_day_max = (df['Close']-next_close).abs().max()

print('Single Day Max:', single_day_max)

avg_daily_volume = df['Traded Volume'].mean()

print('Average Daily Volume:', avg_daily_volume)

median_daily_volume = df['Traded Volume'].median()

print('Median Daily Volume:', median_daily_volume)

highest: 53.54
lowest: 33.62
Max Change: 2.8100000000000023
Single Day Max: 2.559999999999995
Average Daily Volume: 89124.33725490196
Median Daily Volume: 76286.0


In [10]:
#Okay, so now let's do that whole thing over again using dictionaries. 

dct = df.to_dict()

In [11]:
high = max(list(dct['High'].values()))

low = min(list(dct['Low'].values()))

print('High, Low:', high, low)

#So far so good. Now we want max change
max_delta = max([mx - mn for mx, mn in zip(dct['High'].values(), dct['Low'].values())])
print('Max Delta:', max_delta)


#Again so far so good. We've got agreement with what our (infinitely more elegant) Pandas code has
#been telling us. Now we want to find the biggest shift between any two closing prices.

close = list(dct['Close'].values())
biggest_shift = max([abs(nxt_day-day) for day, nxt_day in (zip(close[1:], close[:-1]))])
print('Biggest Shift:', biggest_shift)

#Finally, average daily trading volume.
from statistics import mean, median
volume = list(dct['Traded Volume'].values()) 
mean_volume, median_volume = mean(volume), median(volume)
print('Mean Volume, Median Volume:', mean_volume, median_volume)

#Good. We're all done here.

High, Low: 53.54 33.62
Max Delta: 2.8100000000000023
Biggest Shift: 2.559999999999995
Mean Volume, Median Volume: 89124.33725490196 76286.0
